<a href="https://colab.research.google.com/github/Ayanlola2002/Stutern/blob/master/Stutern_Group_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import pandas as pd
all_accounts=pd.read_csv('/content/drive/My Drive/raw/all_accounts.csv')
all_balances=pd.read_csv('/content/drive/My Drive/raw/all_balances.csv')
all_demographics=pd.read_csv('/content/drive/My Drive/raw/all_demographics.csv',encoding='latin',sep='|')
all_investments=pd.read_csv('/content/drive/My Drive/raw/all_investments.csv')
all_loans=pd.read_csv('/content/drive/My Drive/raw/all_loans.csv')
all_loans_application=pd.read_csv('/content/drive/My Drive/raw/all_loans_application.csv')
all_transactions_digital=pd.read_csv('/content/drive/My Drive/raw/all_transactions_digital.csv')
bureau_score=pd.read_csv('/content/drive/My Drive/raw/bureau_score.csv')
all_dictionary=pd.read_csv('/content/drive/My Drive/raw/dictionary.csv',skiprows=[0])

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (17,19,26,28,30,35,36,37,38,39,61,71,73,74,85,100,103,114,115,125,126) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#we use the all_loans_applications customer id 
#this indicates all those customers that apply for loans 
#it may also be possible that a customer ask for multiple loans
print(all_loans.shape)
print(all_loans_application.shape)

(278505, 10)
(23172, 8)


In [0]:
#setting customerId as index 
all_loans_application.set_index('CUSTOMER_UNIQUE_ID',inplace=True)

In [0]:
#extracting customer ID from  all_loan_applications
important_indexes=all_loans_application.index.unique()

In [0]:
#resetting of all_accounts index
all_accounts.set_index('CUSTOMER_UNIQUE_ID',inplace=True)


In [7]:
all_accounts=all_accounts.loc[important_indexes]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


In [8]:
all_accounts['RECORD_STAT'].value_counts()

O    27281
C     1776
Name: RECORD_STAT, dtype: int64

In [9]:
all_accounts.shape

(29068, 4)

In [0]:
all_accounts.drop(columns='MASKED_ACCOUNT',inplace=True)

In [11]:
#checking different type of currency
all_accounts['CCY'].value_counts()

NGN    26765
USD     2082
GBP      146
EUR       64
Name: CCY, dtype: int64

In [0]:
#function to convert different currency to naira
def con(data):
    data.loc[data.CCY=='USD','ACY_AVL_BAL']=data.loc[data.CCY=='USD','ACY_AVL_BAL'].apply(lambda x: x*365)
    data.loc[data.CCY=='EUR','ACY_AVL_BAL']=data.loc[data.CCY=='EUR','ACY_AVL_BAL'].apply(lambda x: x*398)
    data.loc[data.CCY=='GBP','ACY_AVL_BAL']=data.loc[data.CCY=='GPB','ACY_AVL_BAL'].apply(lambda x: x*467)
    data.loc[data.CCY=='JPY','ACY_AVL_BAL']=data.loc[data.CCY=='JPY','ACY_AVL_BAL'].apply(lambda x: x*3.31)
    return data

In [0]:
#changing all account to naira notes
all_accounts=con(all_accounts)

In [0]:
#convert index to int
all_accounts.index=all_accounts.index.astype('int')

In [0]:
#sort index
all_accounts.sort_index(inplace=True)

In [16]:
#meaning of ACY_AVL_BAL
all_dictionary[all_dictionary['FEATURES']=='ACY_AVL_BAL']

,FEATURES,DESCRIPTION,DATA TYPE,Unnamed: 3,Unnamed: 4
14,ACY_AVL_BAL,Specifies the available balance in the account...,NUMBER,NaN,NaN


In [17]:
#some of the ACY_AVL_BAL IS negative we will assumed it is zero 
all_accounts['ACY_AVL_BAL'].value_counts()

 0.00          12405
-52.50           243
-105.00           69
 950.00           33
 1.00             26
               ...  
 77773.41          1
 4058776.43        1
 125.53            1
 14164.08          1
-145.34            1
Name: ACY_AVL_BAL, Length: 14389, dtype: int64

In [0]:
#for all ACY_AVL_BAL that is negative RECORD STAT IS C we convert to zero so as not to affect our model
all_accounts.loc[(all_accounts.ACY_AVL_BAL!=0) & (all_accounts.RECORD_STAT=="C"),'ACY_AVL_BAL']=0

In [19]:
all_accounts.index.value_counts()

69090004     34
578029002    27
172877006    25
657601000    23
679735005    22
             ..
952341006     1
73406011      1
234473009     1
806118002     1
909990007     1
Name: CUSTOMER_UNIQUE_ID, Length: 17986, dtype: int64

In [20]:
#most of the customer id appears more than ones so we can group by like the example below
all_accounts.loc[69090004]

,RECORD_STAT,ACY_AVL_BAL,CCY
CUSTOMER_UNIQUE_ID,,,
69090004,O,0.00,USD
69090004,C,0.00,NGN
69090004,O,37128.66,NGN
69090004,O,0.00,NGN
69090004,O,28.09,NGN
69090004,O,0.00,NGN
69090004,O,0.00,NGN
69090004,O,1262799.23,NGN
69090004,O,89483.40,USD


In [0]:
#currency was removed because all ACY_AVL_BAL was coverted to same currency naira
all_accounts_data=all_accounts.groupby([all_accounts.index])[['ACY_AVL_BAL']].sum()

In [22]:
all_accounts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29068 entries, 56000 to 999944001
Data columns (total 3 columns):
RECORD_STAT    29057 non-null object
ACY_AVL_BAL    28938 non-null float64
CCY            29057 non-null object
dtypes: float64(1), object(2)
memory usage: 1.5+ MB


In [23]:
#there will be no need to merge CCY column to all_accounts dataframe since we converted all currency to naira.
all_accounts_data.head()

,ACY_AVL_BAL
CUSTOMER_UNIQUE_ID,
56000,136043.48
62005,1541719.95
104008,12364.06
175007,20029.55
200006,61489.56


In [0]:
all_balances.set_index('CUSTOMER_UNIQUE_ID',inplace=True)

In [25]:
#focussing on only those who made loan applications according to all_loan_applications
all_balances=all_balances.loc[important_indexes]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


In [26]:
#checking for null values
all_balances.isnull().sum()

MASKED_ACCOUNT    490
VAL_DT            490
BAL               490
dtype: int64

In [0]:
#trying to select the last date balance of customers who applied for loans
#merger=all_balances.groupby([all_balances.index,all_balances.MASKED_ACCOUNT])[['VAL_DT']].max()

In [0]:
all_balances_data=all_balances.sort_values('VAL_DT').groupby(all_balances.index).tail(1)

In [0]:
all_balances_data.head()

,MASKED_ACCOUNT,VAL_DT,BAL
CUSTOMER_UNIQUE_ID,,,
41981006,00******68,10/1/2019,40821.02
737790008,06******08,10/1/2019,323.75
521640001,00******56,10/1/2019,11974.56
391254009,00******30,10/1/2019,6813.05
657601000,00******15,10/1/2019,14998.88


In [28]:
all_balances_data.isnull().any(axis=1).sum()

4

In [31]:
#drop rows with na values
all_balances_data.dropna(how='any',axis=0)

,MASKED_ACCOUNT,VAL_DT,BAL
CUSTOMER_UNIQUE_ID,,,
41981006,00******68,10/1/2019,40821.02
737790008,06******08,10/1/2019,323.75
521640001,00******56,10/1/2019,11974.56
391254009,00******30,10/1/2019,6813.05
657601000,00******15,10/1/2019,14998.88
...,...,...,...
801451003,07******68,9/9/2019,85273.88
588148009,00******46,9/9/2019,1097.83
13231006,00******65,9/9/2019,40002.64


In [37]:
#rows with null values
all_balances_data.loc[all_balances_data['VAL_DT'].isnull()]

,MASKED_ACCOUNT,VAL_DT,BAL
CUSTOMER_UNIQUE_ID,,,
909766003,NaN,NaN,NaN
297233009,NaN,NaN,NaN
311246013,NaN,NaN,NaN
162998000,NaN,NaN,NaN


In [0]:
#drop row index with null values
all_balances_data=all_balances_data.drop([909766003,297233009,311246013,162998000])

In [39]:
all_balances_data.isna().sum()

MASKED_ACCOUNT    0
VAL_DT            0
BAL               0
dtype: int64

In [0]:
#using regex101 online tool on all_demographics
all_demographics.ADDRESS_LINE3=all_demographics.ADDRESS_LINE3.str.replace(',\s*','')
all_demographics.ADDRESS_LINE3=all_demographics.ADDRESS_LINE3.str.replace('\d*','')

In [42]:
all_demographics.head()

,CUSTOMER_UNIQUE_ID,CUSTOMER_TYPE,ADDRESS_LINE3,ADDRESS_LINE4,COUNTRY,NATIONALITY,LANGUAGE,EXPOSURE_COUNTRY,LOCAL_BRANCH,FROZEN,DECEASED,WHEREABOUTS_UNKNOWN,CUSTOMER_CATEGORY,HO_AC_NO,FX_CUST_CLEAN_RISK_LIMIT,OVERALL_LIMIT,FX_CLEAN_RISK_LIMIT,CREDIT_RATING,LIMIT_CCY,CAS_CUST,LIAB_NODE,SEC_CUST_CLEAN_RISK_LIMIT,SEC_CLEAN_RISK_LIMIT,SEC_CUST_PSTL_RISK_LIMIT,SEC_PSTL_RISK_LIMIT,LIAB_BR,PAST_DUE_FLAG,DEFAULT_MEDIA,SSN,SWIFT_CODE,LOC_CODE,UTILITY_PROVIDER,UTILITY_PROVIDER_ID,RISK_PROFILE,DEBTOR_CATEGORY,UDF_1,UDF_2,UDF_3,UDF_4,UDF_5,...,PINCODE,MFI_CUSTOMER,CUSTOMER_PREFIX,DATE_OF_BIRTH,MINOR,SEX,RESIDENT_STATUS,AGE_PROOF_SUBMITTED,CUST_COMM_MODE,PLACE_OF_BIRTH,BIRTH_COUNTRY,US_RES_STATUS,VST_US_PREV,P_PINCODE,PREF_CONTACT_DT,PREF_CONTACT_TIME,D_PINCODE,EDUCATIONAL_STATUS,MARITAL_STATUS,SPOUSE_NAME,SPOUSE_EMP_STATUS,DEPENDENT_CHILDREN,DEPENDENT_OTHERS,ACCOMODATION,EMPLOYMENT_STATUS,EMPLOYMENT_TENURE,RETIREMENT_AGE,PREV_DESIGNATION,PREV_EMPLOYER,DESIGNATION,SALARY,OTHER_INCOME,RENT,INSURANCE,LOAN_PAYMENT,OTHER_EXPENSES,HOUSE_VALUE,CREDIT_CARDS,E_COUNTRY,CCY_PERS_INCEXP
0,18030000,I,IKOYI,LAGOS,NG,NG,ENG,NG,3,N,N,N,IND,NaN,0.0,0.0,0.0,NaN,NGN,NaN,FCUBSPAR,0.0,0.0,0.0,0.0,3,NaN,MAIL,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,23401,N,MR,1966-08-15 00:00:00,N,M,R,N,E,NaN,NG,N,N,NaN,NaN,0.0,NaN,NaN,M,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,33020001,I,IKORODU,LAGOS,NG,NG,ENG,NG,6,N,N,N,IND,NaN,0.0,0.0,0.0,NaN,NGN,NaN,FCUBSPAR,0.0,0.0,0.0,0.0,6,NaN,MAIL,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,MR,1950-01-01 00:00:00,N,M,R,N,NaN,NaN,NG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,69090004,I,Lagos State,", VICTORIA ISLAND",NG,NG,ENG,NG,14,N,N,N,IND,NaN,0.0,0.0,0.0,NaN,NGN,NaN,FCUBSPAR,0.0,0.0,0.0,0.0,14,NaN,MAIL,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,234,N,Ms.,1970-04-10 00:00:00,N,F,R,N,E,NaN,NG,N,N,NaN,NaN,0.0,NaN,NaN,M,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,81010006,I,SHAGAMU ROADIKORODU,LAGOS,NG,NG,ENG,NG,6,N,N,N,IND,NaN,0.0,0.0,0.0,NaN,NGN,NaN,FCUBSPAR,0.0,0.0,0.0,0.0,6,NaN,MAIL,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,234,NaN,MR,1967-04-26 00:00:00,N,M,R,N,NaN,NaN,NG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,71120007,I,SURULERE,LAGOS,NG,NG,ENG,NG,4,N,N,N,IND,NaN,0.0,0.0,0.0,NaN,NGN,NaN,FCUBSPAR,0.0,0.0,0.0,0.0,4,NaN,MAIL,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,234,N,MR,1965-09-24 00:00:00,N,M,R,N,M,NaN,NG,N,N,NaN,NaN,0.0,NaN,NaN,M,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
#dealing with line 4
all_demographics.ADDRESS_LINE4=all_demographics.ADDRESS_LINE4.str.replace(',\s*','')
all_demographics.ADDRESS_LINE4=all_demographics.ADDRESS_LINE4.str.replace('\d*','')

In [0]:
#to csv
all_demographics.to_csv('all_demographics2.csv')

In [45]:
all_demographics.ADDRESS_LINE4.value_counts().to_dict()

{'': 2695,
 ' ': 26,
 ' ABUJA': 1,
 ' IJEGUN LAGOS': 1,
 ' LAGOS': 2,
 ' OFF POWER ENCOUNTER ROADPHC.': 1,
 ' OGUN STATE': 1,
 ' VICTORIA ISLAND': 1,
 ' ZAMFARA STATE': 1,
 '(COLLECT BY HAND)': 1,
 '(H)': 3,
 '(HOLD ALL CORRESPONDENCES)': 1,
 '(HOLD)': 2,
 '(SEND)': 2,
 '(TO BE COLLECTED)': 3,
 '.': 2,
 'ABA': 1,
 'ABA                 ': 2,
 'ABAI': 1,
 'ABAKALIKI ': 1,
 'ABANDAN': 1,
 'ABARANJE ROADIKOTUN.IKOTUN': 1,
 'ABEOKUTA': 1,
 'ABEOKUTA CLSE AREA  A GARKI': 1,
 'ABEOKUTA.': 1,
 'ABIA': 86,
 'ABUJA': 52,
 'ABUJA ': 1,
 'ABUJA FCT': 1,
 'ABUJA.': 4,
 'AC/O-ONYEMA/UGO': 1,
 'ADAMAWA': 58,
 'ADAMAWA.': 1,
 'ADO EKITI': 2,
 'AGBARA': 1,
 'AGO-OKOTALAGOS.': 1,
 'AJAH LAGOS': 1,
 'AKOKO EDO': 1,
 'AKOWONJO LAGOS': 1,
 'AKURE': 1,
 'AKUTE': 1,
 'AKUTEOGUN STATE.': 1,
 'AKWA  IBOM': 1,
 'AKWA IBOM': 119,
 'AKWA IBOM STATE': 2,
 'AKWA IBOM STATE.': 1,
 'AKWA UBIM': 1,
 'AKWA-IBOM': 4,
 'AKWANGA ': 1,
 'ALAKIAIBADAN': 1,
 'ALLEN': 1,
 'ANAMBRA': 90,
 'ANAMBRA STATE.': 1,
 'ANAMBRA.': 1,
 

In [47]:
all_demographics.head()

,CUSTOMER_UNIQUE_ID,CUSTOMER_TYPE,ADDRESS_LINE3,ADDRESS_LINE4,COUNTRY,NATIONALITY,LANGUAGE,EXPOSURE_COUNTRY,LOCAL_BRANCH,FROZEN,DECEASED,WHEREABOUTS_UNKNOWN,CUSTOMER_CATEGORY,HO_AC_NO,FX_CUST_CLEAN_RISK_LIMIT,OVERALL_LIMIT,FX_CLEAN_RISK_LIMIT,CREDIT_RATING,LIMIT_CCY,CAS_CUST,LIAB_NODE,SEC_CUST_CLEAN_RISK_LIMIT,SEC_CLEAN_RISK_LIMIT,SEC_CUST_PSTL_RISK_LIMIT,SEC_PSTL_RISK_LIMIT,LIAB_BR,PAST_DUE_FLAG,DEFAULT_MEDIA,SSN,SWIFT_CODE,LOC_CODE,UTILITY_PROVIDER,UTILITY_PROVIDER_ID,RISK_PROFILE,DEBTOR_CATEGORY,UDF_1,UDF_2,UDF_3,UDF_4,UDF_5,...,PINCODE,MFI_CUSTOMER,CUSTOMER_PREFIX,DATE_OF_BIRTH,MINOR,SEX,RESIDENT_STATUS,AGE_PROOF_SUBMITTED,CUST_COMM_MODE,PLACE_OF_BIRTH,BIRTH_COUNTRY,US_RES_STATUS,VST_US_PREV,P_PINCODE,PREF_CONTACT_DT,PREF_CONTACT_TIME,D_PINCODE,EDUCATIONAL_STATUS,MARITAL_STATUS,SPOUSE_NAME,SPOUSE_EMP_STATUS,DEPENDENT_CHILDREN,DEPENDENT_OTHERS,ACCOMODATION,EMPLOYMENT_STATUS,EMPLOYMENT_TENURE,RETIREMENT_AGE,PREV_DESIGNATION,PREV_EMPLOYER,DESIGNATION,SALARY,OTHER_INCOME,RENT,INSURANCE,LOAN_PAYMENT,OTHER_EXPENSES,HOUSE_VALUE,CREDIT_CARDS,E_COUNTRY,CCY_PERS_INCEXP
0,18030000,I,IKOYI,LAGOS,NG,NG,ENG,NG,3,N,N,N,IND,NaN,0.0,0.0,0.0,NaN,NGN,NaN,FCUBSPAR,0.0,0.0,0.0,0.0,3,NaN,MAIL,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,23401,N,MR,1966-08-15 00:00:00,N,M,R,N,E,NaN,NG,N,N,NaN,NaN,0.0,NaN,NaN,M,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,33020001,I,IKORODU,LAGOS,NG,NG,ENG,NG,6,N,N,N,IND,NaN,0.0,0.0,0.0,NaN,NGN,NaN,FCUBSPAR,0.0,0.0,0.0,0.0,6,NaN,MAIL,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,MR,1950-01-01 00:00:00,N,M,R,N,NaN,NaN,NG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,69090004,I,Lagos State,VICTORIA ISLAND,NG,NG,ENG,NG,14,N,N,N,IND,NaN,0.0,0.0,0.0,NaN,NGN,NaN,FCUBSPAR,0.0,0.0,0.0,0.0,14,NaN,MAIL,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,234,N,Ms.,1970-04-10 00:00:00,N,F,R,N,E,NaN,NG,N,N,NaN,NaN,0.0,NaN,NaN,M,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,81010006,I,SHAGAMU ROADIKORODU,LAGOS,NG,NG,ENG,NG,6,N,N,N,IND,NaN,0.0,0.0,0.0,NaN,NGN,NaN,FCUBSPAR,0.0,0.0,0.0,0.0,6,NaN,MAIL,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,234,NaN,MR,1967-04-26 00:00:00,N,M,R,N,NaN,NaN,NG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,71120007,I,SURULERE,LAGOS,NG,NG,ENG,NG,4,N,N,N,IND,NaN,0.0,0.0,0.0,NaN,NGN,NaN,FCUBSPAR,0.0,0.0,0.0,0.0,4,NaN,MAIL,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,234,N,MR,1965-09-24 00:00:00,N,M,R,N,M,NaN,NG,N,N,NaN,NaN,0.0,NaN,NaN,M,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
all_demographics.isnull().sum()/len(all_demographics)

CUSTOMER_UNIQUE_ID    0.000000
CUSTOMER_TYPE         0.000000
ADDRESS_LINE3         0.025543
ADDRESS_LINE4         0.151308
COUNTRY               0.000000
                        ...   
OTHER_EXPENSES        0.967501
HOUSE_VALUE           0.967501
CREDIT_CARDS          0.999722
E_COUNTRY             0.987423
CCY_PERS_INCEXP       0.999499
Length: 127, dtype: float64

In [0]:
#removing all columns with columns greater than 90% nan
all_demographics.dropna(thresh=.9*len(all_demographics),axis=1,inplace=True)

In [50]:

all_demographics.columns

Index(['CUSTOMER_UNIQUE_ID', 'CUSTOMER_TYPE', 'ADDRESS_LINE3', 'COUNTRY',
       'NATIONALITY', 'LANGUAGE', 'EXPOSURE_COUNTRY', 'LOCAL_BRANCH', 'FROZEN',
       'DECEASED', 'WHEREABOUTS_UNKNOWN', 'CUSTOMER_CATEGORY', 'LIMIT_CCY',
       'LIAB_NODE', 'LIAB_BR', 'DEFAULT_MEDIA', 'UTILITY_PROVIDER',
       'AML_REQUIRED', 'FT_ACCTING_AS_OF', 'UNADVISED', 'CONSOL_TAX_CERT_REQD',
       'INDIVIDUAL_TAX_CERT_REQD', 'CLS_CCY_ALLOWED', 'CLS_PARTICIPANT',
       'EXT_REF_NO', 'CIF_CREATION_DATE', 'RP_CUSTOMER', 'PRIVATE_CUSTOMER',
       'AUTO_CREATE_ACCOUNT', 'TRACK_LIMITS', 'CUSTOMER_PREFIX',
       'DATE_OF_BIRTH', 'MINOR', 'SEX', 'BIRTH_COUNTRY', 'MARITAL_STATUS'],
      dtype='object')

In [51]:
all_demographics.isnull().sum().to_dict()

{'ADDRESS_LINE3': 459,
 'AML_REQUIRED': 0,
 'AUTO_CREATE_ACCOUNT': 0,
 'BIRTH_COUNTRY': 69,
 'CIF_CREATION_DATE': 5,
 'CLS_CCY_ALLOWED': 0,
 'CLS_PARTICIPANT': 0,
 'CONSOL_TAX_CERT_REQD': 0,
 'COUNTRY': 0,
 'CUSTOMER_CATEGORY': 0,
 'CUSTOMER_PREFIX': 534,
 'CUSTOMER_TYPE': 0,
 'CUSTOMER_UNIQUE_ID': 0,
 'DATE_OF_BIRTH': 5,
 'DECEASED': 0,
 'DEFAULT_MEDIA': 6,
 'EXPOSURE_COUNTRY': 0,
 'EXT_REF_NO': 1600,
 'FROZEN': 0,
 'FT_ACCTING_AS_OF': 4,
 'INDIVIDUAL_TAX_CERT_REQD': 0,
 'LANGUAGE': 0,
 'LIAB_BR': 0,
 'LIAB_NODE': 0,
 'LIMIT_CCY': 5,
 'LOCAL_BRANCH': 0,
 'MARITAL_STATUS': 134,
 'MINOR': 35,
 'NATIONALITY': 0,
 'PRIVATE_CUSTOMER': 510,
 'RP_CUSTOMER': 0,
 'SEX': 20,
 'TRACK_LIMITS': 0,
 'UNADVISED': 0,
 'UTILITY_PROVIDER': 0,
 'WHEREABOUTS_UNKNOWN': 0}

In [0]:
#meaning of minor
all_demographics.MINOR.fillna(all_demographics.MINOR.mode(),inplace=True)

In [54]:
#meaning of ACY_AVL_BAL
all_dictionary[all_dictionary['FEATURES']=='TRACK_LIMITS']

,FEATURES,DESCRIPTION,DATA TYPE,Unnamed: 3,Unnamed: 4


In [0]:
all_demographics.TRACK_LIMITS.fillna("Unknown",inplace=True)

In [0]:
all_demographics.LIMIT_CCY.fillna("Unknown",inplace=True)

In [57]:
all_demographics.isnull().sum()

CUSTOMER_UNIQUE_ID             0
CUSTOMER_TYPE                  0
ADDRESS_LINE3                459
COUNTRY                        0
NATIONALITY                    0
LANGUAGE                       0
EXPOSURE_COUNTRY               0
LOCAL_BRANCH                   0
FROZEN                         0
DECEASED                       0
WHEREABOUTS_UNKNOWN            0
CUSTOMER_CATEGORY              0
LIMIT_CCY                      0
LIAB_NODE                      0
LIAB_BR                        0
DEFAULT_MEDIA                  6
UTILITY_PROVIDER               0
AML_REQUIRED                   0
FT_ACCTING_AS_OF               4
UNADVISED                      0
CONSOL_TAX_CERT_REQD           0
INDIVIDUAL_TAX_CERT_REQD       0
CLS_CCY_ALLOWED                0
CLS_PARTICIPANT                0
EXT_REF_NO                  1600
CIF_CREATION_DATE              5
RP_CUSTOMER                    0
PRIVATE_CUSTOMER             510
AUTO_CREATE_ACCOUNT            0
TRACK_LIMITS                   0
CUSTOMER_P

In [0]:
columns=all_demographics.columns

In [0]:

for i in columns:
    result=all_demographics[i].value_counts().to_list()
    percent=result[0]/sum(result)
    if percent>.9:
        all_demographics.drop(columns=i)

In [61]:
all_demographics.isnull().sum()

CUSTOMER_UNIQUE_ID             0
CUSTOMER_TYPE                  0
ADDRESS_LINE3                459
COUNTRY                        0
NATIONALITY                    0
LANGUAGE                       0
EXPOSURE_COUNTRY               0
LOCAL_BRANCH                   0
FROZEN                         0
DECEASED                       0
WHEREABOUTS_UNKNOWN            0
CUSTOMER_CATEGORY              0
LIMIT_CCY                      0
LIAB_NODE                      0
LIAB_BR                        0
DEFAULT_MEDIA                  6
UTILITY_PROVIDER               0
AML_REQUIRED                   0
FT_ACCTING_AS_OF               4
UNADVISED                      0
CONSOL_TAX_CERT_REQD           0
INDIVIDUAL_TAX_CERT_REQD       0
CLS_CCY_ALLOWED                0
CLS_PARTICIPANT                0
EXT_REF_NO                  1600
CIF_CREATION_DATE              5
RP_CUSTOMER                    0
PRIVATE_CUSTOMER             510
AUTO_CREATE_ACCOUNT            0
TRACK_LIMITS                   0
CUSTOMER_P

In [62]:
all_demographics.head()

,CUSTOMER_UNIQUE_ID,CUSTOMER_TYPE,ADDRESS_LINE3,COUNTRY,NATIONALITY,LANGUAGE,EXPOSURE_COUNTRY,LOCAL_BRANCH,FROZEN,DECEASED,WHEREABOUTS_UNKNOWN,CUSTOMER_CATEGORY,LIMIT_CCY,LIAB_NODE,LIAB_BR,DEFAULT_MEDIA,UTILITY_PROVIDER,AML_REQUIRED,FT_ACCTING_AS_OF,UNADVISED,CONSOL_TAX_CERT_REQD,INDIVIDUAL_TAX_CERT_REQD,CLS_CCY_ALLOWED,CLS_PARTICIPANT,EXT_REF_NO,CIF_CREATION_DATE,RP_CUSTOMER,PRIVATE_CUSTOMER,AUTO_CREATE_ACCOUNT,TRACK_LIMITS,CUSTOMER_PREFIX,DATE_OF_BIRTH,MINOR,SEX,BIRTH_COUNTRY,MARITAL_STATUS
0,18030000,I,IKOYI,NG,NG,ENG,NG,3,N,N,N,IND,NGN,FCUBSPAR,3,MAIL,N,N,M,N,N,N,D,N,183,1999-10-16 00:00:00,N,N,N,Y,MR,1966-08-15 00:00:00,N,M,NG,M
1,33020001,I,IKORODU,NG,NG,ENG,NG,6,N,N,N,IND,NGN,FCUBSPAR,6,MAIL,N,N,M,N,N,N,D,N,1332,1999-10-16 00:00:00,N,N,N,Y,MR,1950-01-01 00:00:00,N,M,NG,S
2,69090004,I,Lagos State,NG,NG,ENG,NG,14,N,N,N,IND,NGN,FCUBSPAR,14,MAIL,N,N,M,N,N,N,D,N,4699,1999-10-16 00:00:00,N,N,N,Y,Ms.,1970-04-10 00:00:00,N,F,NG,M
3,81010006,I,SHAGAMU ROADIKORODU,NG,NG,ENG,NG,6,N,N,N,IND,NGN,FCUBSPAR,6,MAIL,N,N,M,N,N,N,D,N,6811,1999-10-16 00:00:00,N,N,N,Y,MR,1967-04-26 00:00:00,N,M,NG,M
4,71120007,I,SURULERE,NG,NG,ENG,NG,4,N,N,N,IND,NGN,FCUBSPAR,4,MAIL,N,N,M,N,N,N,D,N,17712,1999-11-03 00:00:00,N,N,N,Y,MR,1965-09-24 00:00:00,N,M,NG,M


In [0]:
all_loans_application.index=all_loans_application.index.astype('int')

In [0]:
all_loans_application.reset_index(inplace=True)

In [68]:
all_loans_application.sort_values(['CUSTOMER_UNIQUE_ID','OBLIGOR_RISK_RATING','OVERDUE_DAYS','BOOKING_DATE','MATURITY_DATE','INTEREST_RATE'],inplace=True)

KeyError: ignored

In [69]:
all_loans_application.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23172 entries, 0 to 23171
Data columns (total 8 columns):
CUSTOMER_UNIQUE_ID     23172 non-null int64
OFFERID                23172 non-null object
INCR_ALLOWED           23172 non-null float64
APPLIED_AMOUNT         23172 non-null float64
DATE_OF_APPLY          23172 non-null object
APPLICATION_STATUS     23172 non-null object
MODULE                 23172 non-null object
LOAN_CLASSIFICATION    23172 non-null object
dtypes: float64(2), int64(1), object(5)
memory usage: 1.4+ MB
